#EN3150 Assignment 03: Simple convolutional neural network to perform classification.



##1 .CNN for image classification

### Mount the google Drive

In [83]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Load the Dataset

In [84]:
# Import necessary libraries
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt


# Define the base directory containing your images
base_dir = '/content/drive/My Drive/EN3150-Assignment-03-CNN/Images/'

# Lists to hold image data and labels
image_paths = []
labels = []

# Load images from each folder (H1, H2, H3, H5, H6)
for folder in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder)
    if os.path.isdir(folder_path):
        for filename in os.listdir(folder_path):
            if filename.endswith('.jpg'):
                # Construct full image path
                image_path = os.path.join(folder_path, filename)
                image_paths.append(image_path)
                labels.append(folder)  # Assign label based on folder name

# Convert labels to a NumPy array
labels = np.array(labels)

### Split the Dataset

In [85]:
# Split data: 60% train, 20% validation, 20% test
train_paths, temp_paths, train_labels, temp_labels = train_test_split(
    image_paths, labels, test_size=0.4, random_state=42, stratify=labels
)
validation_paths, test_paths, validation_labels, test_labels = train_test_split(
    temp_paths, temp_labels, test_size=0.5, random_state=42, stratify=temp_labels
)

# Print the sizes of the splits
print(f'Training set: {len(train_paths)} images')
print(f'Validation set: {len(validation_paths)} images')
print(f'Test set: {len(test_paths)} images')

Training set: 5468 images
Validation set: 1823 images
Test set: 1823 images


## Build CNN Model with modifications

In [90]:
from tensorflow.keras import layers, models

# Define the CNN architecture
def build_cnn(x1, m1, x2, m2, x3, x4, m3, d, num_classes):
    model = models.Sequential()

    # First Convolutional Layer
    model.add(layers.Conv2D(x1, (m1, m1), activation='relu', input_shape=(500, 500, 3), name='Conv2D_1'))
    model.add(layers.MaxPooling2D((2, 2), name='MaxPooling2D_1'))

    # Second Convolutional Layer
    model.add(layers.Conv2D(x2, (m2, m2), activation='relu', name='Conv2D_2'))
    model.add(layers.MaxPooling2D((2, 2), name='MaxPooling2D_2'))

    # Third Convolutional Layer (newly added)
    model.add(layers.Conv2D(x4, (m3, m3), activation='relu', name='Conv2D_3'))
    model.add(layers.MaxPooling2D((2, 2), name='MaxPooling2D_3'))

    # Flatten the output
    model.add(layers.Flatten(name='Flatten'))

    # Fully Connected Layer
    model.add(layers.Dense(x3, activation='relu', name='Dense_1'))
    model.add(layers.Dropout(d, name='Dropout_1'))  # Dropout layer to reduce overfitting

    # Additional Fully Connected Layer
    model.add(layers.Dense(x3, activation='relu', name='Dense_2'))
    model.add(layers.Dropout(d, name='Dropout_2'))  # Additional Dropout layer

    # Output Layer
    model.add(layers.Dense(num_classes, activation='softmax', name='Output'))

    return model

# Build and compile the CNN model
cnn_model = build_cnn(x1=32, m1=3, x2=64, m2=3, x3=128, x4=128, m3=3, d=0.5, num_classes=5)
cnn_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the model summary
cnn_model.summary()

Model: "sequential_13"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ Conv2D_1 (Conv2D)                    │ (None, 498, 498, 32)        │             896 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ MaxPooling2D_1 (MaxPooling2D)        │ (None, 249, 249, 32)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_2 (Conv2D)                    │ (None, 247, 247, 64)        │          18,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ MaxPooling2D_2 (MaxPooling2D)        │ (None, 123, 123, 64)        │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Conv2D_3 (Conv2D)                    │ (None, 121, 121, 128)       │          73,856 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ MaxPooling2D_3 (MaxPooling2D)        │ (None, 60, 60, 128)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Flatten (Flatten)                    │ (None, 460800)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dense_1 (Dense)                      │ (None, 128)                 │      58,982,528 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dropout_1 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dense_2 (Dense)                      │ (None, 128)                 │          16,512 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Dropout_2 (Dropout)                  │ (None, 128)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ Output (Dense)                       │ (None, 5)                   │             645 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 59,092,933 (225.42 MB)

 Trainable params: 59,092,933 (225.42 MB)

 Non-trainable params: 0 (0.00 B)